In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import gmaps

# Import the requests library
import requests

# Import the API key
from config import weather_api_key
from config import g_key

# Import the datetime module from the datetime library.
from datetime import datetime

import time

In [2]:
# Store the CSV you saved created in part one into a DataFrame
weather_data_df = pd.read_csv("Weather_Data/WeatherPy_challenge.csv")
weather_data_df.head(10)

,City_ID,City,Lat,Lng,Weather,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Rain Fall,Snow Fall
0,0,Cheremshan,54.67,51.50,few clouds,35.96,77,24,14.67,RU,2020-04-16 22:47:55,0.00,0.00
1,1,Rudbar,30.15,62.60,light rain,61.39,66,73,7.94,AF,2020-04-16 22:47:56,0.44,0.00
2,2,Mahebourg,-20.41,57.70,scattered clouds,75.20,73,40,12.75,MU,2020-04-16 22:47:56,0.00,0.00
3,3,Cape Town,-33.93,18.42,overcast clouds,60.01,93,86,3.36,ZA,2020-04-16 22:47:38,0.00,0.00
4,4,Punta Arenas,-53.15,-70.92,broken clouds,50.00,87,75,8.05,CL,2020-04-16 22:47:56,0.00,0.00
5,5,Vaini,-21.20,-175.20,broken clouds,78.80,78,75,5.82,TO,2020-04-16 22:47:57,0.00,0.00
6,6,Calvinia,-31.47,19.78,few clouds,53.80,37,14,2.37,ZA,2020-04-16 22:43:20,0.00,0.00
7,7,Mataura,-46.19,168.86,moderate rain,46.00,95,100,5.01,NZ,2020-04-16 22:47:57,3.06,0.00
8,8,Ust-Nera,64.57,143.20,light snow,0.12,79,100,11.70,RU,2020-04-16 22:47:57,0.00,0.11
9,9,Kahului,20.89,-156.47,few clouds,82.40,48,20,16.11,US,2020-04-16 22:47:57,0.00,0.00


In [3]:
# Ask the customer to add a minimum and maximum temperature value

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# Ask the customer yes/no questions about rain/snow
yes = {'yes','y', 'ye', '', 'Y', 'YES'}
no = {'no','n', 'N', 'NO'}

input_rain = str(input("Do you want it to be raining? y/n :"))
i=0
while i<1:
    if input_rain in yes:
        rain_bool = True
        i=1
    elif input_rain in no:
        rain_bool = False
        i=1
    else:
        input_rain = str(input("Please resond by 'yes'or 'no', do you want it to be raining? y/n :"))
    
input_snow = str(input("Do you want it to be snowing?y/n :"))
i=0
while i<1:
    if input_snow in yes:
        snow_bool = True
        i=1
    elif input_snow in no:
        snow_bool = False
        i=1
    else:
        input_snow = str(input("Please resond by 'yes'or 'no', do you want it to be snowing? y/n :"))

Do you want it to be raining? y/n :n
Do you want it to be snowing?y/n :n


In [5]:
print(input_rain)
print(rain_bool)
print(input_snow)
print(snow_bool)

n
False
n
False


In [6]:
# Filter the dataset to find the cities that fit the temp criteria
if rain_bool==True:
    rain_statement= weather_data_df["Rain Fall"]>0
else:
    rain_statement= weather_data_df["Rain Fall"]==0

if snow_bool==True:
    snow_statement= weather_data_df["Snow Fall"]>0
else:
    snow_statement= weather_data_df["Snow Fall"]==0    


preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) &
                                        rain_statement & snow_statement]
# Dropping the row with a Nan value
preferred_cities_df=preferred_cities_df.dropna()

preferred_cities_df.head()

,City_ID,City,Lat,Lng,Weather,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Rain Fall,Snow Fall
2,2,Mahebourg,-20.41,57.70,scattered clouds,75.2,73,40,12.75,MU,2020-04-16 22:47:56,0.0,0.0
5,5,Vaini,-21.20,-175.20,broken clouds,78.8,78,75,5.82,TO,2020-04-16 22:47:57,0.0,0.0
9,9,Kahului,20.89,-156.47,few clouds,82.4,48,20,16.11,US,2020-04-16 22:47:57,0.0,0.0
13,13,Kapaa,22.08,-159.32,scattered clouds,80.6,65,40,12.75,US,2020-04-16 22:43:27,0.0,0.0
15,15,Souillac,-20.52,57.52,scattered clouds,75.2,73,40,12.75,MU,2020-04-16 22:47:58,0.0,0.0


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type" : "lodging",
    "key" : g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df.head()

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
2,Mahebourg,MU,75.2,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
5,Vaini,TO,78.8,broken clouds,-21.20,-175.20,Keleti Beach Resort
9,Kahului,US,82.4,few clouds,20.89,-156.47,Maui Seaside Hotel
13,Kapaa,US,80.6,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
15,Souillac,MU,75.2,scattered clouds,-20.52,57.52,Bils' Villa


In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt> Current Weather</dt><dd>{Weather} and {Max Temp}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Create the output file (CSV)
output_data_file = "Weather_Data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV
hotel_df.to_csv(output_data_file, index_label="City_ID")